# Path of Exile Market Analysis

This notebook provides comprehensive analysis of Path of Exile economic data extracted from poe.ninja API.

## Data Sources
- Currency exchange rates
- Skill gems pricing
- Divination cards values
- Unique items pricing

## Analysis Goals
1. Market trend analysis
2. Profit opportunity identification
3. Price volatility assessment
4. Investment recommendations

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print('Libraries imported successfully!')

## Data Loading and Exploration

In [ ]:
# Define data directories
RAW_DATA_DIR = '/opt/airflow/logs/poe_data'
ANALYTICS_DIR = '/opt/airflow/logs/poe_analytics'

# Alternative local paths for development
if not os.path.exists(RAW_DATA_DIR):
    RAW_DATA_DIR = '../logs/poe_data'
    ANALYTICS_DIR = '../logs/poe_analytics'

print(f'Raw data directory: {RAW_DATA_DIR}')
print(f'Analytics directory: {ANALYTICS_DIR}')

# Check if directories exist
print(f'Raw data exists: {os.path.exists(RAW_DATA_DIR)}')
print(f'Analytics data exists: {os.path.exists(ANALYTICS_DIR)}')

In [ ]:
def get_latest_file(directory, prefix):
    """Get the latest file with given prefix from directory"""
    if not os.path.exists(directory):
        return None
    
    files = [f for f in os.listdir(directory) if f.startswith(prefix)]
    if not files:
        return None
    
    return os.path.join(directory, sorted(files)[-1])

def load_json_data(file_path):
    """Load JSON data from file"""
    if not file_path or not os.path.exists(file_path):
        return None
    
    with open(file_path, 'r') as f:
        return json.load(f)

def load_csv_data(file_path):
    """Load CSV data from file"""
    if not file_path or not os.path.exists(file_path):
        return None
    
    return pd.read_csv(file_path)

print('Data loading functions defined!')

## Currency Market Analysis

In [ ]:
# Load currency data
currency_analysis_file = get_latest_file(ANALYTICS_DIR, 'currency_analysis_')
currency_csv_file = get_latest_file(ANALYTICS_DIR, 'currency_data_')

if currency_analysis_file:
    currency_analysis = load_json_data(currency_analysis_file)
    print('Currency analysis data loaded successfully!')
    print(f'File: {currency_analysis_file}')
    
    # Display summary statistics
    if currency_analysis and 'summary_stats' in currency_analysis:
        stats = currency_analysis['summary_stats']
        print(f'\nCurrency Market Summary:')
        print(f'Total currencies tracked: {stats.get("total_currencies", "N/A")}')
        print(f'Average chaos value: {stats.get("avg_chaos_value", "N/A"):.2f}' if stats.get('avg_chaos_value') else 'Average chaos value: N/A')
        print(f'High-value currencies (>100c): {stats.get("high_value_currencies", "N/A")}')
        print(f'Most liquid currency: {stats.get("most_liquid_currency", "N/A")}')
else:
    print('No currency analysis data found. Run the extraction DAG first.')
    currency_analysis = None

In [ ]:
# Load and analyze currency CSV data
if currency_csv_file:
    currency_df = load_csv_data(currency_csv_file)
    
    if currency_df is not None and not currency_df.empty:
        print(f'Currency DataFrame shape: {currency_df.shape}')
        print(f'\nCurrency DataFrame columns: {list(currency_df.columns)}')
        
        # Display first few rows
        print('\nFirst 5 rows:')
        display(currency_df.head())
        
        # Basic statistics
        print('\nBasic statistics:')
        display(currency_df.describe())
    else:
        print('Currency CSV data is empty or could not be loaded')
        currency_df = None
else:
    print('No currency CSV data found')
    currency_df = None

In [ ]:
# Currency market visualization
if currency_df is not None and not currency_df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Currency Market Analysis', fontsize=16, fontweight='bold')
    
    # 1. Top 10 currencies by chaos value
    if 'chaos_equivalent' in currency_df.columns and 'currency_name' in currency_df.columns:
        top_currencies = currency_df.nlargest(10, 'chaos_equivalent')
        axes[0, 0].barh(top_currencies['currency_name'], top_currencies['chaos_equivalent'])
        axes[0, 0].set_title('Top 10 Currencies by Chaos Value')
        axes[0, 0].set_xlabel('Chaos Equivalent')
    
    # 2. Currency category distribution
    if 'currency_category' in currency_df.columns:
        category_counts = currency_df['currency_category'].value_counts()
        axes[0, 1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
        axes[0, 1].set_title('Currency Category Distribution')
    
    # 3. Market liquidity analysis
    if 'listing_count' in currency_df.columns and 'chaos_equivalent' in currency_df.columns:
        axes[1, 0].scatter(currency_df['chaos_equivalent'], currency_df['listing_count'], alpha=0.6)
        axes[1, 0].set_xlabel('Chaos Equivalent')
        axes[1, 0].set_ylabel('Listing Count')
        axes[1, 0].set_title('Value vs Liquidity')
        axes[1, 0].set_xscale('log')
        axes[1, 0].set_yscale('log')
    
    # 4. Price distribution
    if 'chaos_equivalent' in currency_df.columns:
        currency_df['chaos_equivalent'].hist(bins=30, ax=axes[1, 1])
        axes[1, 1].set_xlabel('Chaos Equivalent')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Price Distribution')
        axes[1, 1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()
else:
    print('No currency data available for visualization')

## Skill Gems Profit Analysis

In [ ]:
# Load gems data
gems_analysis_file = get_latest_file(ANALYTICS_DIR, 'gems_analysis_')
gems_csv_file = get_latest_file(ANALYTICS_DIR, 'gems_data_')

if gems_analysis_file:
    gems_analysis = load_json_data(gems_analysis_file)
    print('Gems analysis data loaded successfully!')
    
    # Display top profit opportunities
    if gems_analysis and 'top_profit_opportunities' in gems_analysis:
        print('\nTop 5 Gem Profit Opportunities:')
        opportunities = gems_analysis['top_profit_opportunities'][:5]
        for i, gem in enumerate(opportunities, 1):
            print(f"{i}. {gem.get('name', 'Unknown')}")
            print(f"   Current Value: {gem.get('chaos_value', 0):.1f}c")
            print(f"   Estimated Profit: {gem.get('estimated_leveling_profit', 0):.1f}c")
            print(f"   Profit Margin: {gem.get('profit_margin_percent', 0):.1f}%")
            print()
else:
    print('No gems analysis data found')
    gems_analysis = None

In [ ]:
# Load and analyze gems CSV data
if gems_csv_file:
    gems_df = load_csv_data(gems_csv_file)
    
    if gems_df is not None and not gems_df.empty:
        print(f'Gems DataFrame shape: {gems_df.shape}')
        
        # Filter profitable gems
        if 'estimated_leveling_profit' in gems_df.columns:
            profitable_gems = gems_df[gems_df['estimated_leveling_profit'] > 0]
            print(f'Profitable gems found: {len(profitable_gems)}')
            
            if not profitable_gems.empty:
                print('\nTop 10 Most Profitable Gems:')
                top_profitable = profitable_gems.nlargest(10, 'profit_margin_percent')
                display(top_profitable[['name', 'chaos_value', 'estimated_leveling_profit', 'profit_margin_percent', 'gem_level', 'gem_category']])
    else:
        print('Gems CSV data is empty or could not be loaded')
        gems_df = None
else:
    print('No gems CSV data found')
    gems_df = None

In [ ]:
# Gems profit visualization
if gems_df is not None and not gems_df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Skill Gems Profit Analysis', fontsize=16, fontweight='bold')
    
    # 1. Profit margin distribution
    if 'profit_margin_percent' in gems_df.columns:
        profitable_gems = gems_df[gems_df['profit_margin_percent'] > 0]
        if not profitable_gems.empty:
            profitable_gems['profit_margin_percent'].hist(bins=30, ax=axes[0, 0])
            axes[0, 0].set_xlabel('Profit Margin (%)')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Profit Margin Distribution')
    
    # 2. Gem category profit analysis
    if 'gem_category' in gems_df.columns and 'estimated_leveling_profit' in gems_df.columns:
        category_profit = gems_df.groupby('gem_category')['estimated_leveling_profit'].mean()
        category_profit.plot(kind='bar', ax=axes[0, 1])
        axes[0, 1].set_title('Average Profit by Gem Category')
        axes[0, 1].set_ylabel('Average Profit (chaos)')
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Level vs Value relationship
    if 'gem_level' in gems_df.columns and 'chaos_value' in gems_df.columns:
        axes[1, 0].scatter(gems_df['gem_level'], gems_df['chaos_value'], alpha=0.6)
        axes[1, 0].set_xlabel('Gem Level')
        axes[1, 0].set_ylabel('Chaos Value')
        axes[1, 0].set_title('Gem Level vs Value')
        axes[1, 0].set_yscale('log')
    
    # 4. Top profitable gems
    if 'estimated_leveling_profit' in gems_df.columns and 'name' in gems_df.columns:
        top_profit_gems = gems_df.nlargest(10, 'estimated_leveling_profit')
        axes[1, 1].barh(range(len(top_profit_gems)), top_profit_gems['estimated_leveling_profit'])
        axes[1, 1].set_yticks(range(len(top_profit_gems)))
        axes[1, 1].set_yticklabels([name[:20] + '...' if len(name) > 20 else name for name in top_profit_gems['name']])
        axes[1, 1].set_xlabel('Estimated Profit (chaos)')
        axes[1, 1].set_title('Top 10 Profitable Gems')
    
    plt.tight_layout()
    plt.show()
else:
    print('No gems data available for visualization')

## Divination Cards Investment Analysis

In [ ]:
# Load divination cards data
cards_analysis_file = get_latest_file(ANALYTICS_DIR, 'cards_analysis_')
cards_csv_file = get_latest_file(ANALYTICS_DIR, 'cards_data_')

if cards_analysis_file:
    cards_analysis = load_json_data(cards_analysis_file)
    print('Cards analysis data loaded successfully!')
    
    # Display summary
    if cards_analysis and 'summary_stats' in cards_analysis:
        stats = cards_analysis['summary_stats']
        print(f'\nDivination Cards Market Summary:')
        print(f'Total cards tracked: {stats.get("total_cards", "N/A")}')
        print(f'High-value cards (>50c): {stats.get("high_value_cards", "N/A")}')
        print(f'Most expensive card: {stats.get("most_expensive_card", "N/A")}')
        print(f'Most liquid card: {stats.get("most_liquid_card", "N/A")}')
else:
    print('No cards analysis data found')
    cards_analysis = None

In [ ]:
# Load and analyze cards CSV data
if cards_csv_file:
    cards_df = load_csv_data(cards_csv_file)
    
    if cards_df is not None and not cards_df.empty:
        print(f'Cards DataFrame shape: {cards_df.shape}')
        
        # High-value cards analysis
        if 'chaos_value' in cards_df.columns:
            high_value_cards = cards_df[cards_df['chaos_value'] > 50]
            print(f'High-value cards (>50c): {len(high_value_cards)}')
            
            if not high_value_cards.empty:
                print('\nTop 10 Most Valuable Cards:')
                top_valuable = high_value_cards.nlargest(10, 'chaos_value')
                display(top_valuable[['name', 'chaos_value', 'stack_size', 'value_per_card', 'listing_count']])
    else:
        print('Cards CSV data is empty or could not be loaded')
        cards_df = None
else:
    print('No cards CSV data found')
    cards_df = None

## Market Summary and Recommendations

In [ ]:
# Load market summary
market_summary_file = get_latest_file(ANALYTICS_DIR, 'market_summary_')

if market_summary_file:
    market_summary = load_json_data(market_summary_file)
    print('Market summary loaded successfully!')
    
    # Display comprehensive market overview
    if market_summary:
        print(f'\nMarket Analysis Timestamp: {market_summary.get("timestamp", "N/A")}')
        
        # Currency market overview
        if 'currency_market' in market_summary:
            curr_market = market_summary['currency_market']
            print(f'\n=== CURRENCY MARKET ===')
            print(f'Total currencies: {curr_market.get("total_currencies", "N/A")}')
            print(f'Average value: {curr_market.get("avg_chaos_value", 0):.2f}c')
            print(f'High-value currencies: {curr_market.get("high_value_currencies", "N/A")}')
        
        # Gems market overview
        if 'gems_market' in market_summary:
            gems_market = market_summary['gems_market']
            print(f'\n=== GEMS MARKET ===')
            print(f'Total gems: {gems_market.get("total_gems", "N/A")}')
            print(f'Profitable gems: {gems_market.get("profitable_gems", "N/A")}')
            print(f'High-profit gems (>50% margin): {gems_market.get("high_profit_gems", "N/A")}')
        
        # Cards market overview
        if 'cards_market' in market_summary:
            cards_market = market_summary['cards_market']
            print(f'\n=== DIVINATION CARDS MARKET ===')
            print(f'Total cards: {cards_market.get("total_cards", "N/A")}')
            print(f'High-value cards: {cards_market.get("high_value_cards", "N/A")}')
            print(f'Most expensive: {cards_market.get("most_expensive_card", "N/A")}')
        
        # Top gem opportunities
        if 'top_gem_opportunities' in market_summary:
            print(f'\n=== TOP 5 GEM OPPORTUNITIES ===')
            for i, gem in enumerate(market_summary['top_gem_opportunities'], 1):
                print(f"{i}. {gem.get('name', 'Unknown')} - {gem.get('profit_margin_percent', 0):.1f}% margin")
else:
    print('No market summary data found')
    market_summary = None

In [ ]:
# Generate investment recommendations
print('=== INVESTMENT RECOMMENDATIONS ===\n')

recommendations = []

# Currency recommendations
if currency_df is not None and not currency_df.empty:
    if 'listing_count' in currency_df.columns and 'chaos_equivalent' in currency_df.columns:
        # Find undervalued high-liquidity currencies
        high_liquidity = currency_df[currency_df['listing_count'] > currency_df['listing_count'].quantile(0.7)]
        stable_currencies = high_liquidity[high_liquidity['chaos_equivalent'] > 1]
        
        if not stable_currencies.empty:
            recommendations.append({
                'type': 'Currency Trading',
                'recommendation': f"Focus on {stable_currencies.iloc[0]['currency_name']} - high liquidity with stable value",
                'risk': 'Low',
                'potential_return': 'Steady'
            })

# Gems recommendations
if gems_df is not None and not gems_df.empty:
    if 'profit_margin_percent' in gems_df.columns:
        high_margin_gems = gems_df[gems_df['profit_margin_percent'] > 100]
        
        if not high_margin_gems.empty:
            best_gem = high_margin_gems.iloc[0]
            recommendations.append({
                'type': 'Gem Leveling',
                'recommendation': f"Level {best_gem['name']} - {best_gem['profit_margin_percent']:.1f}% profit margin",
                'risk': 'Medium',
                'potential_return': 'High'
            })

# Cards recommendations
if cards_df is not None and not cards_df.empty:
    if 'chaos_value' in cards_df.columns and 'listing_count' in cards_df.columns:
        valuable_liquid_cards = cards_df[(cards_df['chaos_value'] > 10) & (cards_df['listing_count'] > 5)]
        
        if not valuable_liquid_cards.empty:
            best_card = valuable_liquid_cards.iloc[0]
            recommendations.append({
                'type': 'Card Investment',
                'recommendation': f"Invest in {best_card['name']} - {best_card['chaos_value']:.1f}c value with good liquidity",
                'risk': 'Medium',
                'potential_return': 'Medium'
            })

# Display recommendations
if recommendations:
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['type']}")
        print(f"   Strategy: {rec['recommendation']}")
        print(f"   Risk Level: {rec['risk']}")
        print(f"   Expected Return: {rec['potential_return']}")
        print()
else:
    print('No specific recommendations available. Run the data extraction pipeline first.')

print('\n=== GENERAL TRADING TIPS ===')
print('1. Monitor market trends regularly using this analysis')
print('2. Diversify investments across currencies, gems, and cards')
print('3. Focus on high-liquidity items for quick trades')
print('4. Consider gem leveling for long-term profit')
print('5. Watch for seasonal price fluctuations')

## Data Export and Automation

This section provides utilities for exporting analysis results and setting up automated reporting.

In [ ]:
def export_analysis_report():
    """Export a comprehensive analysis report"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    report_file = f'poe_market_report_{timestamp}.txt'
    
    with open(report_file, 'w') as f:
        f.write('PATH OF EXILE MARKET ANALYSIS REPORT\n')
        f.write('=' * 50 + '\n\n')
        f.write(f'Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n\n')
        
        # Add market summary if available
        if market_summary:
            f.write('MARKET OVERVIEW\n')
            f.write('-' * 20 + '\n')
            
            if 'currency_market' in market_summary:
                curr = market_summary['currency_market']
                f.write(f'Currency Market: {curr.get("total_currencies", "N/A")} currencies tracked\n')
            
            if 'gems_market' in market_summary:
                gems = market_summary['gems_market']
                f.write(f'Gems Market: {gems.get("profitable_gems", "N/A")} profitable opportunities\n')
            
            if 'cards_market' in market_summary:
                cards = market_summary['cards_market']
                f.write(f'Cards Market: {cards.get("high_value_cards", "N/A")} high-value cards\n')
        
        f.write('\nRECOMMENDATIONS\n')
        f.write('-' * 20 + '\n')
        
        if recommendations:
            for i, rec in enumerate(recommendations, 1):
                f.write(f"{i}. {rec['type']}: {rec['recommendation']}\n")
        else:
            f.write('No specific recommendations available.\n')
    
    print(f'Analysis report exported to: {report_file}')
    return report_file

# Export report
report_file = export_analysis_report()
print(f'Report saved as: {report_file}')

## Next Steps

1. **Automated Data Collection**: The Airflow DAGs will automatically collect fresh data every 6 hours
2. **Real-time Monitoring**: Set up alerts for significant market changes
3. **Advanced Analytics**: Implement machine learning models for price prediction
4. **Portfolio Tracking**: Add functionality to track your actual investments
5. **API Integration**: Connect to trading bots for automated execution

### Running the Analysis

To get fresh data and run this analysis:

1. Trigger the `poe_data_extraction` DAG in Airflow
2. Wait for the `poe_data_transformation` DAG to complete
3. Re-run this notebook to see updated analysis

### Accessing Airflow

- Airflow UI: http://localhost:8080 (admin/admin)
- Jupyter Lab: http://localhost:8888 (token: airflow)